In [554]:
# Import Modules
import pandas as pd
import numpy as np

import datetime
import dateparser
from datetime import date, timedelta

# Import Modules
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

import sys
# adding Notebooksfolder to the system path
sys.path.insert(0, '/Users/emil/miniforge3/envs/googleapi/Notebooks')

import importlib
import dates_funcs
importlib.reload(dates_funcs)
from dates_funcs import appendDFToCSV


In [555]:
#FRAN https://www.shortautomaton.com/connecting-to-google-search-console-api-with-python/ 

# Define function to get authorization
def gsc_auth(scopes):
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', scopes)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secrets.json', scopes)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('searchconsole', 'v1', credentials=creds)

    return service

scopes = ['https://www.googleapis.com/auth/webmasters.readonly']

service = gsc_auth(scopes)

Query Function

Find latest Date in old CSV

In [556]:
# Check of Dates
f = open("latestDate.txt")
maxSavedDate = f.read()
f.close()
#print("Previously fetched data up to and including: "+ maxSavedDate)
maxSavedDateDT = datetime.datetime.strptime(maxSavedDate,'%Y-%m-%d').date()
print("Last date saved: "+maxSavedDate)

Last date saved: 2022-12-31


In [557]:
# Now to print to log when program running 
nowDT = datetime.datetime.now()
now = datetime.datetime.strftime(nowDT,'%Y-%m-%d_%H:%M:%S')
print("Script started: "+now)

# Check of Dates
f = open("latestDate.txt")
maxSavedDate = f.read()
f.close()
# print("Previously fetched data up to and including: "+ maxSavedDate)
maxSavedDateDT = datetime.datetime.strptime(maxSavedDate,'%Y-%m-%d').date()

# Todays date
todayDT = date.today()
today = datetime.datetime.strftime(todayDT,'%Y-%m-%d')

# Three days ago to use as end date, since search console is not updated daily
three_days_agoDT = todayDT + datetime.timedelta(days=-3)
three_days_ago = datetime.datetime.strftime(three_days_agoDT,'%Y-%m-%d')

# start_date as the next day as maxSavedDate
test_start_dateDT = maxSavedDateDT + datetime.timedelta(days=1)
test_start_date = datetime.datetime.strftime(test_start_dateDT,'%Y-%m-%d')
print("Fetching new data, starting: "+test_start_date)

# test_end_dateDT = three days ago
test_end_dateDT = three_days_agoDT
test_end_date = datetime.datetime.strftime(test_end_dateDT,'%Y-%m-%d')

print("up to and including: "+test_end_date)

start_date = test_start_date
end_date = test_end_date

Script started: 2023-06-14_15:06:33
Fetching new data, starting: 2023-01-01
up to and including: 2023-06-11


In [453]:
# First Dates with data available

#max_date_overall_name = '2022-01-27'
#max_date_overall = dateparser.parse(max_date_overall_name, settings={'TIMEZONE': 'America/Los_Angeles'}).date()

#test_start_date_name = '2022-01-28'
#test_start_date = dateparser.parse(test_start_date_name, settings={'TIMEZONE': 'America/Los_Angeles'}).date()

#test_end_date_name = 'ten days ago'
#test_end_date = dateparser.parse(test_end_date_name, settings={'TIMEZONE': 'America/Los_Angeles'}).date()



LOOPA TEST!

In [558]:
# Assign Start Row
start_row = 0

# Create empty DataFrame to combine data into
all_search_analytics_df = pd.DataFrame()

# Build Request Body

sa_request = {
            #'startDate': start_date,
            #'endDate': end_date,
            'startDate': start_date,
            'endDate': end_date,
            'dimensions': ["date","page","device","query","country"],
            'rowLimit': 25000,
            }

site_url = "https://advokatfamiljforsvar.se"


# Loop over requests until all rows are pulled into DataFrame
while True:
    sa_request['startRow'] = start_row
    gsc_search_analytics = service.searchanalytics().query(siteUrl=site_url, body=sa_request).execute()

    try:

        all_search_analytics_df = pd.concat([all_search_analytics_df, pd.DataFrame(gsc_search_analytics['rows'])])
    except KeyError:
        break

    if len(gsc_search_analytics['rows']) < 25000:
        break

    start_row += 25000

In [559]:
# Deep copy
df = all_search_analytics_df.copy()

In [560]:
df['index'] = df.reset_index().index
df.set_index('index',inplace=True)

In [561]:
# new df from the column of lists
df_split = pd.DataFrame(df['keys'].tolist(), columns=["date","page","device","query","country"])
# concat df and split_df
df_split['index'] = df_split.reset_index().index
df_split.set_index('index',inplace=True)

df_concat = pd.concat([df, df_split], axis=1)
# display df
df = df_concat.drop('keys', axis=1)

In [562]:
df = pd.concat([df.iloc[:,-5:], df.iloc[:,0:-5]], axis="columns")

In [563]:
df.reset_index(drop=True,inplace=True)

In [564]:
df = dates_funcs.sortOut_date_searchFormat(df)

In [565]:
# Columns to case matching reference and analytics data
df['device'] = df['device'].str.lower()
df['country'] = df['country'].str.upper()
            
# Column names to matching reference and analytics data
df.rename(columns = {'country':'countryIdLong', 'page':'landingPage', 'device':'deviceCategory'}, inplace = True)          

In [566]:
df = df.replace('', np.nan)
print(df.shape)

(324071, 16)


In [567]:
# Split by years
df_by_years = dates_funcs.split_years(df)

In [568]:
df_by_years_dc = df_by_years.copy()

In [569]:
pd.options.mode.chained_assignment = None 

df_by_months = pd.DataFrame()

for x in range(len(df_by_years_dc)):
    df_by_months[x] = []

for x in range(len(df_by_years_dc)):
    df_by_months[x] = dates_funcs.split_months(df_by_years_dc[x])

pd.options.mode.chained_assignment = 'warn' 

In [570]:
# Flatten df_by_months[x][y] (two levels) structure to array of dataframes (one level)
dfs_array = df_by_months.to_numpy().flatten(order='F')

In [571]:
 ## Max time to latestDatefile
    
# Find largest date in dataframes, looking at df eventName, which should have all dates. (?)
maxTimestamp = df["dateFull"].max()
maxTimestampString = datetime.datetime.strftime(maxTimestamp,'%Y-%m-%d')
maxDate = datetime.datetime.strptime(maxTimestampString,'%Y-%m-%d').date()
    
print("Fetched data until "+maxTimestampString)
    
    # If date is larger than priviously max date.
if maxDate > maxSavedDateDT:
    # Write largest date to file
    f = open("latestDate.txt", 'w')
    f.write(maxTimestampString)
    f.close()
    
print("Written to latestDate.txt")
print("\n")

Fetched data until 2023-06-11
Written to latestDate.txt




In [572]:
# For all dataframes, select max year and month for naming.
# Convert dataframes all datatypes to strings as well.
fileNameList = []

for x in range(len(dfs_array)):
    dfs_array[x] = dfs_array[x].astype(str)
    fileNameList.append(dfs_array[x]['year'].min()+'_'+dfs_array[x]['month'].min())

# Convert list of datetime, to list of strings
#fileNameList = [dts.strftime("%Y-%m-%d") for dts in fileNameList]

In [573]:
# Loop over all dataframes and write to file with filename of firstdate in df.
for x, y in zip(range(len(dfs_array)),fileNameList):
    dfs_array[x].to_csv(f'output/csv/search_console_from_{y}.csv',index=None,mode='a')